# Assignment 3: Image Classification

In [1]:
!nvidia-smi

Mon Aug 21 05:12:52 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 532.03                 Driver Version: 532.03       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 L...  WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   57C    P0               10W /  N/A|    204MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# !unzip "/content/drive/MyDrive/Lab Training ML/Assignment 3/food-11.zip"

### Import Packages

In [3]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm
from torchvision.datasets import DatasetFolder, VisionDataset
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset

In [4]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

### Transforms

In [5]:
test_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(90, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
])


### Datasets

In [6]:
class FoodDataset(Dataset):
    def __init__(self, tfm, path="food-11", isTrain=True):
        super(FoodDataset).__init__()
        if isTrain:
            self.train_path = path + "/train"
            self.valid_path = path + "/valid"
            self.files = [self.train_path + "/" + x for x in os.listdir(self.train_path) if x.endswith(".jpg")]
            self.files += [self.valid_path + "/" + x for x in os.listdir(self.valid_path) if x.endswith(".jpg")]
            np.random.shuffle(self.files)
        else:
            self.path = path + "/test"
            self.files = sorted([self.path + "/" + x for x in os.listdir(self.path) if x.endswith(".jpg")])

        self.transform = tfm

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)

        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1  # test has no label

        return im, label


### Models

In [7]:
from torchvision import models


class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 224, 224]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  # [64, 112, 112]
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, 1, 1),  # [128, 112, 112]
            nn.BatchNorm2d(128),
        )
        self.layer2_relu = nn.Sequential(
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  # [128, 56, 56]
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, 3, 1, 1),  # [256, 56, 56]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  # [256, 28, 28]
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(256, 512, 3, 1, 1),  # [512, 28, 28]
            nn.BatchNorm2d(512),
        )
        self.layer4_relu = nn.Sequential(
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  # [512, 14, 14]
        )
        self.layer5 = nn.Sequential(
            nn.Conv2d(512, 512, 3, 1, 1),  # [512, 14, 14]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  # [512, 7, 7]
        )
        self.fc = nn.Sequential(
            nn.Linear(512 * 7 * 7, 1024), 
            nn.ReLU(), 
            nn.Dropout(0.3), 
            nn.Linear(1024, 512), 
            nn.ReLU(), 
            nn.Linear(512, 11),
        )

    def forward(self, x):
        x1 = self.layer1(x)
        residual = x1

        x2 = self.layer2(x1)
        x2 = x2 + residual
        x2 = self.layer2_relu(x2)

        x3 = self.layer3(x2)
        residual = x3

        x4 = self.layer4(x3)
        x4 = x4 + residual
        x4 = self.layer4_relu(x4)

        out = self.layer5(x4)
        out = out.view(out.size()[0], -1)
        return self.fc(out)


class Resnet(nn.Module):
    def __init__(self):
        super(Resnet, self).__init__()
        self.cnn = models.resnet18(weights=None)
        self.cnn.fc = nn.Sequential(nn.Linear(512, 11))

    def forward(self, x):
        return self.cnn(x)


class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.vgg16 = models.vgg16(weights=None)
        self.fc = nn.Sequential(
            nn.ReLU(),
            nn.Linear(1000, 100), 
            nn.ReLU(), 
            nn.Linear(100, 11),
        )

    def forward(self, x):
        return self.vgg16(x)


### Configurations

In [8]:
device =  "cuda" if torch.cuda.is_available() else "cpu"

# hyperparameters
batch_size = 64
n_epochs = 80

patience = 8  # If no improvement in 'patience' epochs, early stop.


### Construct Dataset

In [9]:
dataset = FoodDataset(train_tfm)

### Start Training

In [10]:
from sklearn.model_selection import KFold
from torch.utils.data import SubsetRandomSampler

In [11]:
_exp_name = ["ensemble/sample1", "ensemble/sample2", "ensemble/sample3"]

In [12]:
def adjust_learning_rate(optimizer):
    isPrint = False
    for param_group in optimizer.param_groups:
        if isPrint == False:
            lr = param_group["lr"]
            print(f"--- Learning rate decreases from {lr:.6f} to {lr * 0.8:.6f}. ---")
            isPrint = True
        param_group["lr"] = param_group["lr"] * 0.8


In [13]:
kf = KFold(n_splits=3)

for fold, (train_idx, valid_idx) in enumerate(kf.split(dataset)):
    print(f"Fold {fold+1}/3")
    
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=0, pin_memory=True )
    valid_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=0, pin_memory=True )

    model = Resnet().to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    
    # Initialize trackers, these are not parameters and should not be changed
    stale = 0
    best_acc = 0

    for epoch in range(n_epochs):
        if stale > 5:
            adjust_learning_rate(optimizer)

        # ---------- Training ----------
        model.train()
        train_loss = []
        train_accs = []

        with tqdm(total=len(train_loader), unit="batch") as tqdm_bar:
            tqdm_bar.set_description(f"Epoch {epoch + 1:03d}/{n_epochs:03d}")
            for batch in train_loader:
                imgs, labels = batch

                # Forward the data.
                logits = model(imgs.to(device))

                # Calculate the cross-entropy loss.
                loss = criterion(logits, labels.to(device))

                # Gradients stored in the parameters in the previous step should be cleared out first.
                optimizer.zero_grad()

                # Compute the gradients for parameters.
                loss.backward()

                # Clip the gradient norms for stable training.
                grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

                # Update the parameters with computed gradients.
                optimizer.step()

                # Compute the accuracy for current batch.
                acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

                # Record the loss and accuracy.
                train_loss.append(loss.item())
                train_accs.append(acc)

                tqdm_bar.update(1)
                tqdm_bar.set_postfix(loss=f"{sum(train_loss)/len(train_loss):.5f}", acc=f"{sum(train_accs) / len(train_accs):.5f}", val_loss=f"{0:.5f}", val_acc=f"{0:.5f}")

            train_loss = sum(train_loss) / len(train_loss)
            train_acc = sum(train_accs) / len(train_accs)
            tqdm_bar.set_postfix(loss=f"{train_loss:.5f}", acc=f"{train_acc:.5f}", val_loss=f"{0:.5f}", val_acc=f"{0:.5f}")

            # ---------- Validation ----------
            model.eval()
            valid_loss = []
            valid_accs = []

            for batch in valid_loader:
                imgs, labels = batch

                # Using torch.no_grad() accelerates the forward process.
                with torch.no_grad():
                    logits = model(imgs.to(device))

                # We can still compute the loss (but not the gradient).
                loss = criterion(logits, labels.to(device))

                # Compute the accuracy for current batch.
                acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

                # Record the loss and accuracy.
                valid_loss.append(loss.item())
                valid_accs.append(acc)

                tqdm_bar.set_postfix(
                    loss=f"{train_loss:.5f}", acc=f"{train_acc:.5f}", val_loss=f"{sum(valid_loss) / len(valid_loss):.5f}", val_acc=f"{sum(valid_accs) / len(valid_accs):.5f}"
                )

            # The average loss and accuracy for entire validation set is the average of the recorded values.
            valid_loss = sum(valid_loss) / len(valid_loss)
            valid_acc = sum(valid_accs) / len(valid_accs)

            tqdm_bar.set_postfix(loss=f"{train_loss:.5f}", acc=f"{train_acc:.5f}", val_loss=f"{valid_loss:.5f}", val_acc=f"{valid_acc:.5f}")
            tqdm_bar.close()

        # update logs
        if valid_acc > best_acc:
            with open(f"./{_exp_name[fold]}_log.txt", "a") as f:
                f.write(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
        else:
            with open(f"./{_exp_name[fold]}_log.txt", "a") as f:
                f.write(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

        # save models
        if valid_acc > best_acc:
            print(f"Best model found at epoch {epoch+1}, saving model")
            torch.save(model.state_dict(), f"{_exp_name[fold]}_best.ckpt")  # only save best to prevent output memory exceed error
            best_acc = valid_acc
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"No improvment {patience} consecutive epochs, early stopping")
                break


Fold 1/3


Epoch 001/080: 100%|██████████| 143/143 [02:33<00:00,  1.07s/batch, acc=0.26578, loss=2.08748, val_acc=0.24067, val_loss=2.15899]


Best model found at epoch 1, saving model


Epoch 003/080: 100%|██████████| 143/143 [02:27<00:00,  1.03s/batch, acc=0.36200, loss=1.81207, val_acc=0.30035, val_loss=2.19358]


Best model found at epoch 3, saving model


...

Epoch 077/080: 100%|██████████| 143/143 [02:30<00:00,  1.05s/batch, acc=0.91936, loss=0.23420, val_acc=0.78168, val_loss=0.79091]


Best model found at epoch 77, saving model


Epoch 078/080: 100%|██████████| 143/143 [02:29<00:00,  1.04s/batch, acc=0.92450, loss=0.22457, val_acc=0.79008, val_loss=0.76941]


Best model found at epoch 78, saving model


Epoch 080/080: 100%|██████████| 143/143 [02:28<00:00,  1.04s/batch, acc=0.93116, loss=0.20040, val_acc=0.77619, val_loss=0.82401]


### Dataloader for test

In [25]:
# Construct test datasets.
test_set = FoodDataset(test_tfm, isTrain=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

### Test Time Augmentation

In [26]:
tta_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(90, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
])

tta_num = 5

### Testing and generate prediction CSV

In [42]:
model_1 = Resnet().to(device)
model_1.load_state_dict(torch.load("ensemble/sample1_best.ckpt"))
model_1.eval()

model_2 = Resnet().to(device)
model_2.load_state_dict(torch.load("ensemble/sample2_best.ckpt"))
model_2.eval()

model_3 = Resnet().to(device)
model_3.load_state_dict(torch.load("ensemble/sample3_best.ckpt"))
model_3.eval()

prediction = []
with torch.no_grad():
    for data, _ in tqdm(test_loader):
        for img in data:
            test_input = img.view(1, 3, 224, 224)
            test_input = test_input.to(device)

            test_pred_1 = model_1(test_input)
            test_pred_2 = model_2(test_input)
            test_pred_3 = model_3(test_input)

            test_pred_1 = test_pred_1.cpu().data.numpy()
            test_pred_2 = test_pred_2.cpu().data.numpy()
            test_pred_3 = test_pred_3.cpu().data.numpy()

            # test time augmentation
            tta_pred_1 = np.zeros((1, 11))
            tta_pred_2 = np.zeros((1, 11))
            tta_pred_3 = np.zeros((1, 11))

            for _ in range(tta_num):
                test_augmented = tta_transform(img)
                test_augmented = test_augmented.view(1, 3, 224, 224)
                test_augmented = test_augmented.to(device)

                pred_1 = model_1(test_augmented)
                pred_2 = model_2(test_augmented)
                pred_3 = model_3(test_augmented)

                tta_pred_1 = tta_pred_1 + pred_1.cpu().data.numpy()
                tta_pred_2 = tta_pred_2 + pred_2.cpu().data.numpy()
                tta_pred_3 = tta_pred_3 + pred_3.cpu().data.numpy()

            tta_pred_1 = tta_pred_1 / tta_num
            tta_pred_2 = tta_pred_2 / tta_num
            tta_pred_3 = tta_pred_3 / tta_num

            # final prediction
            test_label = np.argmax((test_pred_1 * 0.7 + tta_pred_1 * 0.3) + (test_pred_2 * 0.7 + tta_pred_2 * 0.3) + (test_pred_3 * 0.7 + tta_pred_3 * 0.3))

            prediction.append(test_label)


100%|██████████| 47/47 [06:30<00:00,  8.32s/it]


In [43]:
# create test csv
def pad4(i):
    return "0" * (4 - len(str(i))) + str(i)


df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(len(test_set))]
df["Category"] = prediction
df.to_csv("submission.csv", index=False)


### Visual Representations Implementation

In [29]:
# import torch
# import numpy as np
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt
# from tqdm import tqdm
# import matplotlib.cm as cm
# import torch.nn as nn

# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# # Load the trained model
# model = Resnet().to(device)
# state_dict = torch.load(f"sample_best.ckpt")
# model.load_state_dict(state_dict)
# model.eval()

# print(model)

In [30]:
# # Load the vaildation set defined by TA
# valid_set = FoodDataset("food-11/valid", tfm=test_tfm)
# valid_loader = DataLoader(valid_set, batch_size=64, shuffle=False, num_workers=0, pin_memory=True)

# # Extract the representations for the specific layer of model
# index = 1 # You should find out the index of layer which is defined as "top" or 'mid' layer of your model.
# features = []
# labels = []
# for batch in tqdm(valid_loader):
#     imgs, lbls = batch
#     with torch.no_grad():
#         logits = model.cnn[:index](imgs.to(device))
#         logits = logits.view(logits.size()[0], -1)
#     labels.extend(lbls.cpu().numpy())
#     logits = np.squeeze(logits.cpu().numpy())
#     features.extend(logits)

# features = np.array(features)
# colors_per_class = cm.rainbow(np.linspace(0, 1, 11))

# # Apply t-SNE to the features
# features_tsne = TSNE(n_components=2, init='pca', random_state=42).fit_transform(features)

# # Plot the t-SNE visualization
# plt.figure(figsize=(10, 8))
# for label in np.unique(labels):
#     plt.scatter(features_tsne[labels == label, 0], features_tsne[labels == label, 1], label=label, s=5)
# plt.legend()
# plt.show()

# plt.figure(figsize=(10, 8))
# labels = [0]
# for label in np.unique(labels):
#     plt.scatter(features_tsne[labels == label, 0], features_tsne[labels == label, 1], label=label, s=5)
# plt.legend()
# plt.show()